In [1]:
!pip install together
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.1 MB/s eta 0:00:00


In [2]:
from together import Together # Using TogetherAI to access Meta-Llama-3.1-405B-Instruct-Turbo
from groq import Groq # Using Groq to access llama3-70b-8192
import os

In [14]:
# Setting TOGETHER_API_KEY
os.environ["TOGETHER_API_KEY"]="f06967a7297cb1dbf70c5d3efa9552fd542a71e2b02d20bd3641493a74384ae3"
os.environ["GROQ_API_KEY"]="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"

In [4]:
# Defining Prompts for Department, Priority and Language Predictions
# System prompt for the model and few shot examples based on train set
system_prompt = """You are acting as a support agent responsible for routing IT support tickets to the appropriate departments.
You will be provided with 7 tickets at a time and each ticket consists of ticket ID, subject and body.

Each support ticket is represented by two pieces of information:
1. Ticket Subject: The title or heading of the support request. This usually gives a brief summary of the issue, often in the form of a question or statement.
2. Ticket Body: The detailed description of the issue or request provided by the customer. This provides further context, describing the nature of the problem, request, or inquiry.

Your goal is to accurately predict the following attributes for each ticket:

1. **Department**: Identify the responsible department. Possible departments include:

    You need to analyze both the ticket subject and ticket body together and classify the ticket into one of the following department categories:

    Department Categories:
    Technical Support: Tickets related to technical issues, such as software bugs, system errors, or malfunctioning hardware.
    Customer Service: Tickets involving customer inquiries related to service quality, user experience, or feedback.
    Billing and Payments: Tickets related to payment issues, invoices, refunds, or subscription management.
    Product Support: Tickets regarding product-related inquiries, issues with specific features, or product installation and configuration.
    IT Support: Tickets related to IT infrastructure, network issues, system access problems, or internal technical support.
    Returns and Exchanges: Tickets related to product returns, exchanges, or refund requests.
    Sales and Pre-Sales: Tickets concerning sales inquiries, product information requests, quotes, or questions about making a purchase.
    Human Resources: Tickets related to HR inquiries, employee benefits, payroll questions, or job-related issues.
    Service Outages and Maintenance: Tickets that refer to service downtimes, scheduled maintenance, or system outages affecting the operation.
    General Inquiry: Tickets that don’t fit into any of the above categories, such as general questions or unrelated issues.

    Instructions for Classification:
    Carefully read both the ticket subject and ticket body.
    Based on the information provided, determine which department would be responsible for handling the issue described in the ticket.
    Choose the department that best matches the content of the ticket. The department should be selected based on the nature of the issue or request, as described in the ticket.

    For example:
    If the subject mentions something like “refund request” and the body explains issues with payment, the correct classification would be Billing and Payments.
    If the subject describes a software bug and the body provides details about how the system is malfunctioning, the correct department would be Technical Support.
    If the subject refers to something like “job application” or “employee benefits,” the department would be Human Resources.

    Expected Output:
    Please provide the department name where this ticket should be routed, based on the content analysis.

    Example of the prompt in action:
    Input:
    Ticket Subject: "Issue with Product Features and Configuration"
    Ticket Body: "I am having trouble configuring my new product. It seems like there is a problem with the installation, and the features are not working as expected."

    Expected Classification:
    Department: Product Support

2. **Priority**: Determine the urgency level of the ticket. Possible priority levels are:
    low:
    These tickets represent non-critical issues that are not urgent and do not require immediate attention.
    The issues may include things like general inquiries, minor feature requests, or issues that are not time-sensitive.
    Example: A question about an account feature that doesn't affect the customer's immediate ability to use the service.

    medium:
    These tickets are moderately urgent. The issue may be important but is not affecting the business or customer in a critical manner.
    It could involve things like delayed responses, issues that need attention within a few hours or by the end of the business day, but are not breaking or disabling core functionalities.
    Example: A customer unable to access certain features of a product, but they can still use some parts of the service.

    high:
    These tickets represent urgent issues that need to be resolved as soon as possible.
    These could include system outages, critical bugs, or problems that significantly impact the customer’s ability to use the product or service.
    High urgency tickets are often time-sensitive and require immediate attention or resolution within a few minutes to hours.

    Example: A service outage, a system crash, or an issue where the customer is unable to use the product at all.

    Instructions for Classification:
    Carefully read both the ticket subject and ticket body.
    Based on the information provided, determine how urgent the issue is and classify the ticket accordingly.
    Pay attention to the following indicators that may help you classify the urgency:
    Keywords or phrases indicating urgency, such as "urgent," "immediate," "down," "problem," or "critical."
    Time sensitivity, such as mentions of deadlines or needing resolution soon.

    Impact on the customer: Is the issue preventing the customer from using the product or service? If yes, it’s likely a High urgency.
    Scope of the problem: If the issue affects a wide range of users or a critical function, it might be High priority. If the issue only affects one user or a non-essential feature, it might be Low.

    Expected Output:
    Please provide the urgency level for this ticket based on the analysis of its subject and body. The possible output values are:
    Low
    Medium
    High

    Example of the prompt in action:
    Input:
    Ticket Subject: "Unable to Access Account - Critical Issue"
    Ticket Body: "I have been trying to log into my account for the past few hours, and the system keeps showing an error message. I urgently need access to my account to complete my work."

    Expected Classification:
    Urgency Level: High

    Another Example:
    Input:
    Ticket Subject: "Query Regarding Billing Information"
    Ticket Body: "I noticed an unexpected charge on my account. Could you provide details about this charge? It's not urgent, but I would like to understand it better."

    Expected Classification:
    Urgency Level: Low

    Key Considerations:
    Urgency Based on Time Sensitivity: The primary factor is how urgent the situation is based on the content of the ticket. Is the issue affecting the customer’s ability to use the product or service immediately?
    Impact on Service: The more critical the issue (e.g., affecting multiple users, service outages), the higher the urgency level should be classified.
    Customer's Tone and Language: Pay attention to the customer’s tone, as words like "urgent," "immediate," or "critical" often indicate a High urgency.

3. **Language**: Identify the language of the ticket. Use the appropriate language code from the following list:
		 You will need to classify the language using ISO 639-1 language codes. These codes consist of two-letter codes assigned to each language. For example:
			Afrikaans (af)
			Albanian (sq)
			Amharic (am)
			Arabic (ar)
			Armenian (hy)
			Azerbaijani (az)
			Bengali (bn)
			Bosnian (bs)
			Bulgarian (bg)
			Catalan (ca)
			Chinese (Simplified) (zh)
			Chinese (Traditional) (zh-TW)
			Croatian (hr)
			Czech (cs)
			Danish (da)
			Dari (fa-AF)
			Dutch (nl)
			English (en)
			Estonian (et)
			Farsi (Persian) (fa)
			Filipino, Tagalog (tl)
			Finnish (fi)
			French (fr)
			French (Canada) (fr-CA)
			Georgian (ka)
			German (de)
			Greek (el)
			Gujarati (gu)
			Haitian Creole (ht)
			Hausa (ha)
			Hebrew (he)
			Hindi (hi)
			Hungarian (hu)
			Icelandic (is)
			Indonesian (id)
			Irish (ga)
			Italian (it)
			Japanese (ja)
			Kannada (kn)
			Kazakh (kk)
			Korean (ko)
			Latvian (lv)
			Lithuanian (lt)
			Macedonian (mk)
			Malay (ms)
			Malayalam (ml)
			Maltese (mt)
			Marathi (mr)
			Mongolian (mn)
			Norwegian (Bokmål) (no)
			Pashto (ps)
			Polish (pl)
			Portuguese (Brazil) (pt)
			Portuguese (Portugal) (pt-PT)
			Punjabi (pa)
			Romanian (ro)
			Russian (ru)
			Serbian (sr)
			Sinhala (si)
			Slovak (sk)
			Slovenian (sl)
			Somali (so)
			Spanish (es)
			Spanish (Mexico) (es-MX)
			Swahili (sw)
			Swedish (sv)
			Tamil (ta)
			Telugu (te)
			Thai (th)
			Turkish (tr)
			Ukrainian (uk)
			Urdu (ur)
			Uzbek (uz)
			Vietnamese (vi)
			Welsh (cy)

    Instructions for Language Identification:
    Carefully read both the ticket subject and ticket body.
    The language code should be based on the overall language used in the text. If the text is mixed with multiple languages, choose the language that is most predominant.
    For common language variants, select the appropriate language code (e.g., use en for English, es for Spanish, etc.). If the language is a variant of a more widely spoken language (e.g., pt-BR for Brazilian Portuguese), select pt.
    If you are unsure, pick the language that is most commonly associated with the language and culture of the text.

    Expected Output:
    You should output the ISO 639-1 language code corresponding to the language in which the ticket is written. The language code should be a two-letter code, such as:
    en (for English)
    de (for German)
    es (for Spanish)
    fr (for French)
    it (for Italian)
    ja (for Japanese)
    ko (for Korean)
    zh (for Chinese)

    Example of the Prompt in Action:
    Example 1:
    Ticket Subject: "Problemas con la conexión a internet" Ticket Body: "Tengo problemas para conectarme a internet, ¿pueden ayudarme?"
    Expected Classification: Language Code: es (Spanish)

    Example 2:
    Ticket Subject: "Can't access my account" Ticket Body: "I have tried resetting my password, but I am still unable to log in. Please help!"
    Expected Classification: Language Code: en (English)

    Example 3:
    Ticket Subject: "Problème de connexion avec le serveur" Ticket Body: "Je ne peux pas me connecter au serveur, cela fait 30 minutes que j'essaie."
    Expected Classification: Language Code: fr (French)

    Key Considerations:
    Accuracy: Be sure to determine the correct language by analyzing the text as a whole, not just isolated keywords. Sometimes, the ticket may contain regional variations (e.g., en-GB for British English or en-US for American English), but you should return the primary code (in this case, en).

Ensure that each ticket is classified accurately based on its content. Use the subject and body of the ticket to make your predictions.
**Examples**:

	1. **Ticket ID**: 1001
		**Subject**: Discrepancia de facturación en Google Workspace
		**Body**: "Estimado equipo de soporte de TI, Estoy escribiendo para informar un monto de facturación incorrecto en mi suscripción de Google Workspace Business Standard bajo la cuenta <acc_num>. Por favor, revise y ajuste la factura. Espero su pronta respuesta. Saludos, "
		**Department**: Billing and Payments
		**Type**: Incident
		**Priority**: low
		**Language**: es

	2. **Ticket ID**: 1002
		**Subject**: Urgent Consultation Request for Critical IT Issues
		**Body**: "Dear IT Services Support Team, I hope this message finds you well. My name is and I am currently experiencing critical issues with our server administration, which are significantly impacting our operations. We rely heavily on your IT Consulting Service for our ongoing technical needs. Therefore, we urgently need your expert assistance to resolve these issues as quickly as possible. Time is of the essence because our operations have come to a standstill. Please contact us at your earliest convenience, either via email or phone at <tel_num>. The swift resolution of this situation is imperative to minimize any further disruption to our business. Thank you for your prompt attention to this matter. Kind regards, "
		**epartment**: Customer Service
		**Type**: Request
		**Priority**: high
		**Language**: en

	3. **Ticket ID**: 1003
		**Subject**: Consulta sobre Servicios de Consultoría en TI
		**Body**: "Estimado Servicio de Atención al Cliente, Mi nombre es , y estoy interesado en obtener más información sobre sus Servicios de Consultoría en TI, centrándome específicamente en el desarrollo de software y la administración de servidores. Agradecería información detallada sobre los servicios ofrecidos, precios y proceso de compromiso. ¿Podría proporcionarme esta información o direccionarme a un representante que pueda ayudarme con mi consulta? Gracias por su asistencia. Atentamente, <tel_num> <email_address>"
		**Department**: General Inquiry
		**Type**: Request
		**Priority**: medium
		**Language**: es

	4. **Ticket ID**: 1004
		**Subject**: Demande de mise à jour des dossiers
		**Body**: "Cher service client, Je vous écris pour demander une mise à jour de notre système de gestion des dossiers des employés en accord avec nos opérations de service de conseil en informatique en cours. Le système actuel nécessite des améliorations pour maximiser l'efficacité et garantir une intégration harmonieuse avec notre infrastructure informatique existante. Dans le cadre de cette demande, nous souhaiterions discuter de l'étendue des services et de toute exigence supplémentaire nécessaire à la mise en œuvre de ces mises à jour. Veuillez me faire savoir un moment approprié pour une discussion détaillée. Nous vous remercions de votre assistance rapide dans cette affaire. Merci, <acc_num>"
		**Department**: Human Resources
		**Type**: Change
		**Priority**: low
		**Language**: fr

	5. **Ticket ID**: 1005
		**Subject**: Issues with Slack connection affecting team communication today
		**Body**: "Dear Customer Support Team, I am encountering challenges when attempting to activate my Kaspersky Internet Security 2024 product. Despite adhering to the suggested guidelines, I am still facing complications. My account number is <acc_num>. Could you kindly assist me with the activation process at your earliest opportunity? You can reach me at <tel_num> or through email at . Thank you for your swift support. Sincerely, "
		**Department**: Product Support
		**Type**: Problem
		**Priority**: medium
		**Language**: en

	6. **Ticket ID**: 1006
		**Subject**: Defective Dell XPS 13 9310
		**Body**: "Dear Tech Online Store Support, I received a Dell XPS 13 9310 with a defective display. Please arrange an exchange at the earliest convenience. My order number is <order_num>. Feel free to contact me at or <tel_num> if you need further information. Best, "
		**Department**: Returns and Exchanges
		**Type**: Incident
		**Priority**: low
		**Language**: en

	7. **Ticket ID**: 1007
		**Subject**: Touchscreen and Keyboard Issues with Surface Pro 7
		**Body**: "Dear Tech Online Store Customer Support, I hope this message finds you well. I am writing to bring to your attention some issues I have been experiencing with my recently purchased Microsoft Surface Pro 7. The main problems concern the touchscreen response and the detachable keyboard connection. Since purchase, the touchscreen functionality has been inconsistent, often failing to register my touch inputs accurately. This inconsistency significantly affects the usability of the device, as I primarily rely on the touch feature for my day-to-day tasks. Furthermore, the detachable keyboard frequently disconnects from the device at random intervals. I have tried several troubleshooting steps including resetting the device, updating drivers, and even reattaching the keyboard several times per day. Despite these efforts, the problem persists. Given the challenges detailed above, I am kindly requesting assistance in resolving these issues. I trust that you may have encountered similar problems and possibly have solutions or processes in place to address them effectively. Please let me know how to proceed or if a replacement is necessary. Thank you for your attention to this matter. Best regards, "
		**Department**: Sales and Pre-Sales
		**Type**: Problem
		**Priority**: medium
		**Language**: en

	8. **Ticket ID**: 1008
		**Subject**: AWS-Serverausfall
		**Body**: "Sehr geehrte Kundenbetreuung, wir hatten einen unerwarteten Ausfall unserer AWS-Server, was zu erheblichen Konnektivitätsproblemen geführt hat. Die Störung hat die Datenabfrage und die Sichtbarkeit der Server für unseren AWS-Managementdienst beeinträchtigt. Sofortige Unterstützung ist dringend erforderlich. Danke, "
		**Department**: Service Outages and Maintenance
		**Type**: Incident
		**Priority**: high
		**Language**: de

	9. **Ticket ID**: 1009
		**Subject**: Solicitud urgente de orientación y solución de problemas
		**Body**: "Estimado soporte al cliente, espero que este mensaje le encuentre bien. Me pongo en contacto para solicitar urgentemente su experiencia en orientación sobre desarrollo de software y solución de problemas del servidor como parte de nuestro actual proyecto de consultoría de TI. Hemos encontrado algunos problemas críticos que están afectando negativamente nuestros plazos del proyecto. Específicamente, nuestro equipo está enfrentando desafíos con la integración del nuevo software con nuestros sistemas existentes, lo que ha llevado a tiempos de inactividad inesperados y retrasos en el rendimiento. Además, ha habido problemas persistentes de conectividad del servidor que no podemos resolver por nuestra cuenta, obstaculizando la entrega fluida de nuestros servicios. Su pronta asistencia en el diagnóstico y rectificación de estos problemas sería inmensamente beneficiosa. Confiamos en su amplia experiencia y destreza técnica para ayudarnos a superar estos obstáculos rápidamente. ¿Podría también organizar una consulta con su desarrollador líder a la mayor brevedad posible? Nuestro objetivo es resolver estos problemas lo más rápido posible para minimizar cualquier retraso adicional en el proyecto y garantizar una experiencia fluida para el cliente. Gracias por su atención inmediata a este asunto. Espero su respuesta rápida. Saludos cordiales, "
		**Department**: Technical Support
		**Type**: Request
		**Priority**: high
		**Language**: es

    **Output Format:**

		For each ticket, provide the output *only* in a JSON dictionary format like this:

		```
		[
			{"ticket_ID": 1001, "Department": "Billing and Payments", "Priority": "Low", "Language": "es"},
			{"ticket_ID": 1002, "Department": "Customer Service", "Priority": "High", "Language": "en"},
			... // more tickets
		]
		```

		Ensure the entire output is a valid JSON array of dictionaries.  Do not include any other text or explanations.
		"""

In [5]:
# Defining Prompts for Type

def generate_type_prompt():
    prompt = """
    You are acting as a support agent responsible for determining the type of an IT support ticket. The goal is to classify the ticket into one of four predefined types based on the ticket subject and ticket body. Your classification should be based on the content and nature of the ticket described in the subject and body.
    You will be provided with 7 tickets at a time and each ticket consists of ticket ID, subject and body.

    Each ticket contains two pieces of information:
    1. Ticket Subject: A brief summary or heading of the issue or request. The subject gives a short overview of the problem or inquiry and usually provides an indication of the ticket type.
    2. Ticket Body: A detailed description of the issue or request. This section provides more context and elaborates on the problem or inquiry.

    Ticket Types:
    The ticket needs to be classified into one of the following ticket types:
    1. Incident:
    Definition: An Incident refers to an unplanned disruption or reduction in the quality of service that is affecting the customer’s ability to use the product or service.
    Characteristics:
    The issue is unexpected and is causing interruption to normal service.
    The incident may be resolved by restoring the service or providing a workaround.
    Example: "The website is down and customers cannot make purchases," or "My email service is not working, I cannot send or receive emails."
    Keywords: "down," "not working," "outage," "failure," "crash," "unable to access," "problem with"

    2. Request:
    Definition: A Request refers to a customer asking for something specific that does not require troubleshooting or issue resolution.
    Characteristics:
    The customer is making a formal request for a service, information, or a change that they need.
    The issue is not urgent or critical, and the service is functioning normally.
    Example: "Can you help me reset my password?" or "I need to update my account details."
    Keywords: "request," "help," "need," "change," "assistance," "modify," "update"

    3. Problem:
    Definition: A Problem refers to a recurring issue or underlying cause of multiple incidents. It is often the root cause of incidents that need investigation or resolution.
    Characteristics:
    The problem is often related to an underlying system issue or defect.
    A Problem might require a long-term solution to fix, preventing future incidents.
    Example: "There’s an issue with the database server that causes intermittent slowdowns," or "A bug in the app keeps crashing on Android."
    Keywords: "bug," "issue," "root cause," "defect," "repeated," "investigation," "error," "failure"

    4. Change:
    Definition: A Change refers to any planned modification or enhancement to a product or system. It is not related to an issue but represents an action taken to improve or modify the system or service.
    Characteristics:
    The request is about upgrading, modifying, or changing a system or service.
    The change is usually planned and can involve system updates, feature additions, or infrastructure upgrades.
    Example: "We need to upgrade the server to increase capacity," or "I want to change the language settings on my account."
    Keywords: "upgrade," "enhance," "modify," "change," "add," "install," "update"

    Instructions for Classification:
    Carefully read both the ticket subject and ticket body.
    Based on the content of the subject and body, determine whether the ticket corresponds to an Incident, Request, Problem, or Change.
    Pay attention to keywords in the text that may indicate the type of ticket:
    Look for words related to service disruption or failure (which could indicate an Incident).
    Look for words like "request," "need," "help," or "change" (which could indicate a Request).
    Look for terms like "bug," "defect," "root cause," or "issue" (which could indicate a Problem).
    Look for words like "upgrade," "modify," or "install" (which could indicate a Change).

    Expected Output:
    You should provide the ticket type based on your classification:
    Incident
    Request
    Problem
    Change

    Example of the Prompt in Action:
    Example 1:
    Ticket Subject: "Website is down, cannot make purchases" Ticket Body: "The website is not loading for me. I have tried on multiple browsers and devices, but the page just doesn’t open."
    Expected Classification: Incident (The website is down, causing disruption to the customer’s ability to make purchases)

    Example 2:
    Ticket Subject: "Need to update my billing information" Ticket Body: "I would like to update my billing details on my account. Can you help me with that?"
    Expected Classification: Request (The customer is requesting an update to their billing information)

    Example 3:
    Ticket Subject: "Recurring bug in mobile app crashes" Ticket Body: "The app crashes every time I try to upload a photo. This happens multiple times per day."
    Expected Classification: Problem (This indicates a recurring bug in the app that causes crashes)

    Example 4:
    Ticket Subject: "Upgrade required for server capacity" Ticket Body: "We need to upgrade our server to handle more traffic. Please advise on the best approach for this."
    Expected Classification: Change (The customer is asking for a change to upgrade the server for more capacity)

    Final Format for Model Output:
    The output should be the ticket type as one of the following:
    Incident
    Request
    Problem
    Change

    Key Considerations:
    Incident: Focus on whether there’s a service disruption that affects the customer’s ability to use the product or service.
    Request: Focus on whether the customer is simply asking for something or requesting a service change without a problem.
    Problem: Focus on whether there’s an underlying issue or recurring pattern that causes incidents.
    Change: Focus on whether the ticket describes a planned modification to the system, such as upgrades or updates.

     **Output Format:**
		For each ticket, provide the output *only* in a JSON dictionary format like this and output should be *only* in English laguage:

		```
		[
			{"ticket_ID": 1001, "Type": "Incident"},
			{"ticket_ID": 1002, "Type": "Request"},
			... // more tickets
		]
		```

		Ensure the entire output is a valid JSON array of dictionaries.  Do not include any other text or explanations.
    """
    return prompt

In [15]:
# Function to send user query to LLM Meta-Llama-3.1-405B
model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo"

def classify_ticket(messages,model=model):
    client=Together()
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

In [16]:
# Using llama3-70b-8192 for Type Predictions
def get_type_prediction(input_message):
  client = Groq()
  completion = client.chat.completions.create(
      model="llama3-70b-8192",
      messages=input_message,
      temperature=0,
      max_completion_tokens=1024,
      top_p=1,
      stream=False,
      stop=None,
  )
  return completion.choices[0].message.content

In [17]:
import csv
import json
import time

In [18]:
input_file = '/content/sample_data/test.csv'
output_file = 'classified_tickets.csv'

# Process 7 records at a time
batch_size = 7

def process_batch(batch, writer):
    user_prompts = []
    ticket_ids = []

    for row in batch:
        ticket_id = row['ticket_ID']
        ticket_subject = row['ticket_subject']
        ticket_body = row['ticket_body']
        user_prompt = f"**ticket_ID**\n\t{ticket_id}\n\n**ticket_subject**\n\t{ticket_subject}\n\n**ticket_body**\n\t{ticket_body}"

        user_prompts.append(user_prompt)
        ticket_ids.append(ticket_id)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "\n\n".join(user_prompts)}
    ]

    messages_type = [
        {"role": "system", "content": generate_type_prompt()},
        {"role": "user", "content": "\n\n".join(user_prompts)}
    ]

    output = classify_ticket(messages)
    print(f"output is {output}")
    output_type = get_type_prediction(messages_type)
    print(f"output is {output_type}")
    responses = output.split('}')[:-1]
    #print(f"responses is {responses}\n")
    responses_type = output_type.split('}')[:-1]
    #print(f"responses_type is {responses_type}\n")

    # Extracting required fields from the model generated ouput
    for i, (response, response_type) in enumerate(zip(responses, responses_type)):
        response = response + '}'
        start_index = response.find('{')
        end_index = response.rfind('}') + 1
        dictionary_str = response[start_index:end_index]

        response_type = response_type + '}'
        start_index_type = response_type.find('{')
        end_index_type = response_type.rfind('}') + 1
        dictionary_str_type = response_type[start_index_type:end_index_type]
        try:
            result = json.loads(dictionary_str)
            result_type = json.loads(dictionary_str_type)
            writer.writerow({
                'ticket_ID': result.get('ticket_ID', 'N/A'),
                'department': result.get('Department', 'N/A'),
                'type': result_type.get('Type', 'N/A'),
                'priority': result.get('Priority', 'N/A').lower(),
                'language': result.get('Language', 'N/A')
            })
        except json.JSONDecodeError:
            writer.writerow({
                'ticket_ID': ticket_ids[i]
            })

with open(input_file, mode='r', encoding='utf-8') as infile, open(output_file, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['ticket_ID', 'department', 'type', 'priority', 'language']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)

    writer.writeheader()

    batch = []
    batch_no = 0
    for i, row in enumerate(reader):
        #if i < 455:  # Skip the first 251 rows
            #continue
        batch.append(row)
        if len(batch) == batch_size:
            batch_no = batch_no + 1
            print(f"Processing batch with size {batch_size} and batch no is {batch_no}")
            process_batch(batch, writer)
            print("Sleeping in between batches")
            time.sleep(5)
            batch = []

    # Process any remaining rows in the last batch
    if batch:
        print("Processing batch")
        process_batch(batch, writer)

print(f"Classified tickets have been saved to {output_file}.")

Processing batch with size 7 and batch no is 1
output is [
  {"ticket_ID": 456, "Department": "Product Support", "Priority": "Low", "Language": "fr"},
  {"ticket_ID": 457, "Department": "Technical Support", "Priority": "Medium", "Language": "pt"},
  {"ticket_ID": 458, "Department": "Product Support", "Priority": "High", "Language": "es"},
  {"ticket_ID": 459, "Department": "Returns and Exchanges", "Priority": "High", "Language": "pt"},
  {"ticket_ID": 460, "Department": "Product Support", "Priority": "Low", "Language": "de"},
  {"ticket_ID": 461, "Department": "Service Outages and Maintenance", "Priority": "Low", "Language": "hi"},
  {"ticket_ID": 462, "Department": "Product Support", "Priority": "Low", "Language": "hi"}
]
output is [
    {"ticket_ID": 456, "Type": "Request"},
    {"ticket_ID": 457, "Type": "Problem"},
    {"ticket_ID": 458, "Type": "Incident"},
    {"ticket_ID": 459, "Type": "Request"},
    {"ticket_ID": 460, "Type": "Incident"},
    {"ticket_ID": 461, "Type": "Reques